# Langchain Agents

[LangChain Agent](https://docs.langchain.com/oss/python/langchain/agents)

Combine LLM with tools to create systems that can reason about tasks decide which tools to use and iteratively work toward solutions. Basically achieve intelligent responses to requests.

A flow (multi-step) that is controlled/defined by calls to an LLM. Allows tools (web access, DB access, humans) to be called before and after LLM.

Kinds of agent
- routers (simple) low control from LLM
- fully autonomous or select based on reasoning a step, can generate its own next move. Reliability drops.

![Agent Control Level](./AgentControlLevel.png)

Agents run in a loop to achieve a goal until a stop condition is met, a final output is emitted or iteration limit reached.

![Agent Loop](./AgentLoop.png)


## Features

* [Dynamic Model Selection](#Dynamic-Model-Selection)
* [Tool Calling](#tool-calling)
* [Middleware](#middleware) - used to
* Dynamic Prompts
* Structured Outputs
* Memory
* 


In [ ]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.chat_models import init_chat_model
from langchain_lib.settings import settings  # library that reads settings from a json settings file

load_dotenv()

def get_chat(model_name: str | None) -> BaseChatModel:
    """Return the chat model.
    
    Args:
        model_name: Name of the model to use. If None, use default from settings.json
    """
    if not model_name:
        llm_model = settings.llm_settings.get('model')
        return init_chat_model(
            model=f"ollama:{llm_model.split('ollama:')[1]}",
            temperature=0.0,
            timeout=10
        )
    else:
        return init_chat_model(
            model=model_name,
            temperature=0.0,
            timeout=10
        )

# using model instance
model = get_chat("ollama:qwen3-coder:30b")
agent = create_agent(model=model, tools=tools)

## Dynamic Model Selection

Some models might be good for functionality like tool calling and others good for reasoning or summarization.

### Method

This is done in middleware from `import langchain.agents.middleware` using a `@wrap_model_call` decorator. When the model is created the middleware function is massed. `middleware=[dynamic_model_selection]`


In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatOpenAI(model="gpt-4o-mini")
advanced_model = ChatOpenAI(model="gpt-4o")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

agent = create_agent(
    model=basic_model,  # Default model
    tools=tools,
    middleware=[dynamic_model_selection]
)

## Tool Calling

These give the agent the ability to take actions like:

* access a database
* do a web search
* access an API

### Method
A function list is passed to `create_agent` function. The functions are decoorated with the `@tool` decorator from `import langchain.tools import tool`.

Error handling is done by middleware and `@wrap_tool_call` decorator

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model=model, # can be the name of a model or an instance
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)

## Middleware

Enables the customization of agent behaviour. Used to

* Process state before the model is called (e.g., message trimming, context injection)
* Modify or validate the model’s response (e.g., guardrails, content filtering)
* Handle tool execution errors with custom logic
* Implement dynamic model selection based on state or context
* Add custom logging, monitoring, or analytics

### Hooks

Specified with a decorator at specific execution points.
* `before_agent` (once per invocation)
* `after_agent`
* `before_model`
* `after_model` (once per invocation)

- `wrap_model_call` - Around each model call
- `wrap_tool_call` - Around each tool call
